ОБУЧИЛ МОДЕЛЬ НА ВСЕХ ТОМАХ "ВОЙНА И МИР" Л.ТОЛСТОГО

In [136]:
import tensorflow as tf

import numpy as np
import os
import time

In [137]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
HW_9.ipynb  less_9.ipynb  voyna-i-mir.txt


In [138]:
%cd gdrive/My\ Drive/nero_final/less_9

[Errno 2] No such file or directory: 'gdrive/My Drive/nero_final/less_9'
/content/gdrive/My Drive/nero_final/less_9


In [139]:
!ls

HW_9.ipynb  less_9.ipynb  voyna-i-mir.txt


In [140]:
path_to_file = 'voyna-i-mir.txt'

In [141]:
text_all = open(path_to_file, 'rb').read().decode(encoding='Windows-1251')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text_all)))

Length of text: 2997950 characters


In [142]:
print(text_all[850:1350])

гаю,]  садитесь и рассказывайте.
Так говорила в июле 1805 года известная Анна Павловна Шерер, фрейлина и приближенная императрицы Марии Феодоровны, встречая важного и чиновного князя Василия, первого приехавшего на ее вечер. Анна Павловна кашляла несколько дней, у нее был грипп , как она говорила (грипп был тогда новое слово, употреблявшееся только редкими). В записочках, разосланных утром с красным лакеем, было написано без различия во всех:
"Si vous n'avez rien de mieux a faire, M. le comte 


In [143]:
import re

#оставим предложения только с русскими словами, без лишней пунктуации

text_s = text_all.split(' ')
text = ''
list_del_el = ['.',',','!','?','...']
punkt_flag = False
for string in text_s:
    match = re.search(r'([?!,.а-яА-ЯёЁ\s]+)', string)
    if match:
        new_string = string[match.start():match.end()]
        if punkt_flag and new_string in list_del_el:
          continue
        else:
          if new_string in list_del_el:
            text = text + new_string
          else:
            text = text + ' ' + new_string 
        punkt_flag = (new_string in list_del_el)

print(text[850:1350])

 во всех, или, .
 Если вас, граф или князь нет в виду ничего лучшего и если перспектива вечера у бедной больной не слишком вас пугает, то я буду очень рада видеть вас нынче у себя между семью и десятью часами. Анна Шерер. 
, О! какое жестокое нападение! отвечал, нисколько не смутясь такою встречей, вошедший князь, в придворном, шитом мундире, в чулках, башмаках, при звездах, с светлым выражением плоского лица. Он говорил на том изысканном французском языке, на котором не только говорили, но и 


In [144]:
print('Length of text: {} characters'.format(len(text)))

Length of text: 90205 characters


In [145]:
text = text + text

In [146]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

66 unique characters


In [147]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [148]:
text_as_int, text, len(text_as_int), len(text)

(array([ 2, 12,  4, ..., 35, 38, 44]),
 ' Е, поместья,. мой верный раб,. Ну, что, князь, Генуа и Лукка стали не больше, как поместьями фамилии Бонапарте. Нет, я вас предупреждаю, если вы мне не скажете, что у нас война, если вы еще позволите себе защищать все гадости, все ужасы этого Антихриста право, я верю, что он Антихрист я вас больше не знаю, вы уж не друг мой, вы уж не мой верный раб, как вы говорите. Ну, здравствуйте, здравствуйте., Я вижу, что я вас пугаю, садитесь и рассказывайте.\r\nТак говорила в июле года известная Анна Павловна Шерер, фрейлина и приближенная императрицы Марии Феодоровны, встречая важного и чиновного князя Василия, первого приехавшего на ее вечер. Анна Павловна кашляла несколько дней, у нее был грипп, как она говорила грипп был тогда новое слово, употреблявшееся только редкими В записочках, разосланных утром с красным лакеем, было написано без различия во всех, или, .\r\n Если вас, граф или князь нет в виду ничего лучшего и если перспектива вечера у бедной 

### train and target

In [149]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

 
Е
,
 
п


In [150]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

' Е, поместья,. мой верный раб,. Ну, что, князь, Генуа и Лукка стали не больше, как поместьями фамилии'
' Бонапарте. Нет, я вас предупреждаю, если вы мне не скажете, что у нас война, если вы еще позволите с'
'ебе защищать все гадости, все ужасы этого Антихриста право, я верю, что он Антихрист я вас больше не '
'знаю, вы уж не друг мой, вы уж не мой верный раб, как вы говорите. Ну, здравствуйте, здравствуйте., Я'
' вижу, что я вас пугаю, садитесь и рассказывайте.\r\nТак говорила в июле года известная Анна Павловна Ш'


In [151]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [152]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  ' Е, поместья,. мой верный раб,. Ну, что, князь, Генуа и Лукка стали не больше, как поместьями фамили'
Target data: 'Е, поместья,. мой верный раб,. Ну, что, князь, Генуа и Лукка стали не больше, как поместьями фамилии'


In [153]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [154]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [155]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [156]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [157]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [158]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 128)         8448      
                                                                 
 gru_3 (GRU)                 (None, None, 1024)        3545088   
                                                                 
 gru_4 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_5 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense_4 (Dense)             (None, None, 66)          67650     
                                                                 
Total params: 16,216,386
Trainable params: 16,216,386
Non-trainable params: 0
_________________________________________________________________


In [159]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 66), dtype=float32, numpy=
array([[-1.9710093e-04, -6.7738455e-04,  2.6661693e-04, ...,
        -1.3337514e-03, -9.9860597e-04,  1.1420939e-03],
       [-9.9837501e-04, -2.4321589e-03,  1.9656348e-03, ...,
        -1.3855344e-03, -1.9900994e-03,  8.7757315e-04],
       [-1.4654356e-03, -3.3540230e-03,  3.5745068e-03, ...,
        -1.0128587e-03, -1.7535152e-03,  2.9360096e-04],
       ...,
       [-2.2209680e-03,  1.0011507e-05,  2.0207292e-03, ...,
         5.8244704e-04,  1.6048349e-03,  2.9517335e-03],
       [-1.8303839e-03,  1.2897085e-03,  1.2170416e-03, ...,
         1.4079099e-03,  1.6180295e-03,  7.8449177e-04],
       [-1.4931470e-03,  2.0083315e-03,  9.2856248e-04, ...,
         2.1838602e-03,  1.9588829e-03, -2.9250316e-04]], dtype=float32)>

In [160]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [161]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ь? сказала Анна Павловна.\r\n, сказал Болконский, ударяя на последнем слоге, как француз,... Генералу '

Next Char Predictions: 
 'ЕАЕ,СкСиФъЭуХТюТжщмфёЧИКТХЧбвлЗулЯхдыцтьЛНЖмэ.екъ,ёУъХНждКщГжтъфд Цщжъ,ФФЯХцг.ачМСшЛрМъсяБ\n.ЗГ\rдюццм'


## Train the model

In [162]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.189642


In [163]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [164]:
!rm -rf ./training_checkpoints

In [165]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [166]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

### Execute the training

In [167]:
EPOCHS = 20

In [168]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
27/27 [==============================] - 610s 22s/step - loss: 3.8318
Epoch 2/20
27/27 [==============================] - 606s 22s/step - loss: 3.1433
Epoch 3/20
27/27 [==============================] - 604s 22s/step - loss: 2.8506
Epoch 4/20
27/27 [==============================] - 595s 22s/step - loss: 2.6570
Epoch 5/20
27/27 [==============================] - 596s 22s/step - loss: 2.4866
Epoch 6/20
27/27 [==============================] - 601s 22s/step - loss: 2.3244
Epoch 7/20
27/27 [==============================] - 602s 22s/step - loss: 2.1611
Epoch 8/20
27/27 [==============================] - 598s 22s/step - loss: 1.9891
Epoch 9/20
27/27 [==============================] - 604s 22s/step - loss: 1.8246
Epoch 10/20
27/27 [==============================] - 609s 23s/step - loss: 1.6550
Epoch 11/20
27/27 [==============================] - 606s 22s/step - loss: 1.4988
Epoch 12/20
27/27 [==============================] - 601s 22s/step - loss: 1.3394
Epoch 13/20
27/27 [======

## Generate text

In [169]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [170]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [171]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 128)         8448      
                                                                 
 gru_6 (GRU)                 (None, None, 1024)        3545088   
                                                                 
 gru_7 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_8 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense_5 (Dense)             (None, None, 66)          67650     
                                                                 
Total params: 16,216,386
Trainable params: 16,216,386
Non-trainable params: 0
_________________________________________________________________


In [186]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 100

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [187]:
text_ = generate_text(model, start_string=u"Если вас, граф или князь нет ")
print(text_)

Если вас, граф или князь нет вопротме. же овераза ологотрь, Пь, кралахо мяноми ноже? Паза ня влотеся ов вуля, и, ба илей пого ось


In [188]:
len(text_)

129

In [189]:
len(text)

180410

**ОБУЧИЛ ВСЕГО НА ПЕРВЫХ 100 000 символов, так как полный объем colab готов был обучать более суток. Думаю, если обучить на всех 4 томах, результат будет лучше)**